In [2]:
import re
import ast
from csv import reader
def load_inputs(path):
    with open('input_puzzles.csv', 'r') as f:
        csv_reader = reader(f)
        list_rows = list(csv_reader)
        input_list = []
        for row in list_rows[1:]:
            initial = parse_input(row[0].strip())
            goal = parse_input(row[1].strip())
            t = [initial, goal]
            input_list.append(t)
    return input_list

def parse_input(x):
    x = re.sub('\(', '[', x)
    x = re.sub('\)', ']', x)
    x = re.sub(';', ',', x)
    x = ast.literal_eval(x)
    return x

def load_puzzles(path):
    with open('input_puzzles_only.csv', 'r') as f:
        csv_reader = reader(f)
        list_rows = list(csv_reader)
        input_list = []
        for row in list_rows[1:]:
            initial = parse_input(row[0].strip())
            input_list.append(initial)
    return input_list

In [26]:
# == test load_inputs() function ===
path = 'input_puzzles.csv'
input_pzs = load_inputs(path)
s0, g = input_pzs[5]
print(s0)
print(g)

[[6, 1, 2], [7, 3, 8], [5, 4, 9]]
[[1, 2, 3], [4, 5, 6], [7, 8, 9]]


In [5]:
# == test load_puzzles() function ===
path = 'input_puzzles_only.csv'
list_inputs = load_puzzles(path)
s0 = list_inputs[5]
print(s0)

[[6, 1, 2], [7, 3, 8], [5, 4, 9]]


In [3]:
def output_one(f, s):
    f.write("(")
    for index in range(len(s)):
        f.write("(")
        for i in range(len(s[index])):
            f.write("{}".format(s[index][i]))
            if i < len(s[index])-1:
                f.write(';')
        if index < len(s)-1:
            f.write(");")
        else:
            f.write(")")
    f.write(")")

def output_all(path, lst):
    with open(path, 'w') as file_obj:
        for i in range(len(lst)):
            output_one(file_obj, lst[i])
            if i < len(lst) - 1:
                file_obj.write('=>')

In [4]:
# == test output_all() function ===
path1 = 'outputs.txt'
result_list = [s0, g]
output_all(path1, result_list)

In [1]:
def mv_R(puzzle,r,c):
    pz = puzzle.copy()
    N = len(puzzle[0])
    if c < N-1:
        pz[r][c], pz[r][c+1] = pz[r][c+1], pz[r][c]
        return pz

def mv_D(puzzle,r,c):
    pz = puzzle.copy()
    N = len(puzzle[0])
    if r < N-1:
        pz[r][c], pz[r+1][c] = pz[r+1][c], pz[r][c]
        return pz

def mv_L(puzzle,r,c):
    pz = puzzle.copy()
    if c > 0:
        pz[r][c], pz[r][c-1] = pz[r][c-1], pz[r][c]
        return pz

def mv_U(puzzle,r,c):
    pz = puzzle.copy()
    if r > 0:
        pz[r][c], pz[r-1][c] = pz[r-1][c], pz[r][c]
        return pz

In [44]:
# == test move function ====
ss = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
print(ss)
mr = mv_R(ss, 0,2)
print(mr)
md = mv_D(ss,2,0)
print(md)
ml = mv_L(ss,0,2)
print(ml)
mu = mv_U(ss, 0, 1)
print(mu)

[[1, 2, 3], [4, 5, 6], [7, 8, 9]]
None
None
[[1, 3, 2], [4, 5, 6], [7, 8, 9]]
None


In [45]:
from anytree import Node, RenderTree
from anytree.exporter import DotExporter
from treelib import Node, Tree

In [46]:
tree = Tree()
tree.create_node("CEO","CEO") #root
tree.create_node("VP_1","VP_1",parent="CEO" )
tree.create_node("VP_2","VP_2",parent="CEO" )
tree.create_node("GM_1","GM_1",parent="VP_1" )
tree.create_node("GM_2","GM_2",parent="VP_2" )
tree.show()

CEO
├── VP_1
│   └── GM_1
└── VP_2
    └── GM_2



In [2]:
import time
class Node_:
    def __init__(self, puzzle, parent, depth, heuristic):
        self.puzzle = puzzle
        self.parent = parent
        self.depth = depth
        self.heuristic = heuristic

def create_node(puzzle, parent, depth, heuristic):
    return Node_(puzzle, parent, depth, heuristic)

In [3]:
def findChildNodes(parent_node):
    puzzle = parent_node.puzzle
    rows = len(puzzle)
    columns = len(puzzle[0])
    child_nodes = []
    for row in range(rows):
        for column in range(columns):
            child_node = create_node(mv_U(puzzle, row, column), parent_node, parent_node.depth + 1, 0)
            child_nodes.append(child_node)
            child_node = create_node(mv_D(puzzle, row, column), parent_node, parent_node.depth + 1, 0)
            child_nodes.append(child_node)
            child_node = create_node(mv_L(puzzle, row, column), parent_node, parent_node.depth + 1, 0)
            child_nodes.append(child_node)
            child_node = create_node(mv_R(puzzle, row, column), parent_node, parent_node.depth + 1, 0)
            child_nodes.append(child_node)
    child_nodes = [node for node in child_nodes if node.puzzle is not None]
    print("Total", len(child_nodes), "possibilities")
    return child_nodes

In [4]:
def isSamePuzzle(s, g):
    for i in range(len(s)):
        for j in range(len(s[0])):
            if s[i][j] != g[i][j]:
                return False
    return True

In [5]:
def isSameNode(n1, n2):
    return isSamePuzzle(n1.puzzle, n2.puzzle)

In [6]:
def DFS(start, goal):
    openList = []
    closeList = []
    depth_limit = 100
    start_node = create_node(start,None,0,0)
    openList.append(start_node)
    start_time = time.time()
    while len(openList) > 0:
        elapsed_time = time.time() - start_time
        if elapsed_time < 60000:
            # get left most node
            current_node = openList.pop(0)
            if isSamePuzzle(current_node.puzzle, goal):
                print(current_node.puzzle)
                temp = current_node
                print(temp.depth)
                return 'Find solution'
            else:
                child_nodes = findChildNodes(current_node)
                closeList.append(current_node)
                # discard child node if already in open or close list
                for node in child_nodes:
                    if node.depth > depth_limit:
                        return 'No solution: exceed depth limit'
                    if len(openList) > 0:
                        for n1 in openList:
                            if isSameNode(n1, node):
                                child_nodes.remove(node)
                                break
                for node in child_nodes:
                    for n2 in closeList:
                        if isSameNode(n2, node):
                            child_nodes.remove(node)
                            break
                    openList.insert(1, node)
                child_nodes = [nd for nd in child_nodes if nd is not None]
        else:
            return 'No solutions: over time'



In [15]:
import itertools
def index_2d(list, v):
    for i, x in enumerate(list):
        if v in x:
            return i, x.index(v)

def in_good_position(x, g, v):
    r, c = index_2d(x, v)
    r_g, c_g = index_2d(g, v)
    return r == r_g and c == c_g

In [8]:
def DFS_1(start, goal):
    openList = []
    closeList = []
    depth_limit = 100
    start_node = create_node(start,None,0,0)
    openList.append(start_node)
    start_time = time.time()
    while len(openList) > 0:
        elapsed_time = time.time() - start_time
        if elapsed_time < 60000:
            # get left most node
            current_node = openList.pop(0)
            if isSamePuzzle(current_node.puzzle, goal):
                print(current_node.puzzle)
                temp = current_node
                print(temp.depth)
                return 'Find solution'
            else:
                for v in list(itertools.chain.from_iterable(goal)):
                    if in_good_position(current_node.puzzle, goal, v):
                        continue
                    else:
                        pass
                    #TODO: add child nodes in openList
                    



        else:
            return 'No solutions: time exceed limit'

In [19]:
s0 = [[6, 1, 2], [7, 8, 3], [5, 4, 9]]
g1 = [[2, 1, 3], [4, 5, 6], [7, 8, 9]]
g2 = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
g = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
# DFS_1(g2, g)
r, c = index_2d(g2, 8)
list(itertools.chain.from_iterable(g1))

[2, 1, 3, 4, 5, 6, 7, 8, 9]